Create noise channel(s)

In [2]:
from qtensor.noise_simulator.NoiseChannels import DepolarizingChannel
from qtensor.noise_simulator.NoiseModel import NoiseModel

# Depolarizing channel for a single qubit. The probability of the error occuring is related to 0.003
depol_chan_1Q = DepolarizingChannel(param = 0.003, num_qubits = 1)
# Depolarizing channel for two qubits. The probability of the error occuring is related to 0.03
depol_chan_2Q = DepolarizingChannel(param = 0.03, num_qubits = 2)

Create noise model

In [3]:
noise_model = NoiseModel()
# If a circuit uses this noise model, all H, XPhase, and ZPhase gates will experience depolarizing noise 
# based on the single qubit depolarizing channel created above
noise_model.add_channel_to_all_qubits(channel = depol_chan_1Q, gates = ['H', 'XPhase', 'ZPhase'])
# If a circuit uses this noise model, all cX gates will experience depolarizing noise 
# based on the two-qubit depolarizing channel created above
noise_model.add_channel_to_all_qubits(channel = depol_chan_2Q, gates = ['cX'])

Create a circuit. Here we create a QAOA circuit. 

In [7]:
from qtree.operators import from_qiskit_circuit
from qtensor import QiskitQAOAComposer, QtreeSimulator
import qtensor
import numpy as np
import networkx as nx

# degree of graph
d = 3
num_qubits = 4
G = nx.random_regular_graph(d, num_qubits)

# optimal gamma and beta parameters previously found for these values of d and n
gammabeta = np.array(qtensor.tools.BETHE_QAOA_VALUES[str(d)]['angles'])
gamma = -gammabeta[:d]
beta = gammabeta[d:]

# composes a qaoa circuit of type qiskit
qiskit_comp = QiskitQAOAComposer(G, gamma=gamma, beta=beta)
qiskit_comp.ansatz_state()

# converts the circuit to qtensor type, which is just a list 
n, circ = from_qiskit_circuit(qiskit_comp.circuit)
# from_qiskiit_circuit returns each gate as a list so we remove the lists from the gates. Might want to fix that.
circ = sum(circ, [])

Instatiate simulator with noise model and then simulate the circuit with noise. 

In [22]:
from qtensor.noise_simulator.NoiseSimulator import NoiseSimulator

noise_sim = NoiseSimulator(noise_model)

num_circs = 10
# simulate an ensemble of 10 circuits with stochastic noise
noise_sim.simulate_batch_ensemble(circ, num_circs, num_qubits)
# get the approximate noisy probabilities 
qtensor_probs = noise_sim.normalized_ensemble_probs
qtensor_probs

array([0.14060763, 0.0349968 , 0.04040751, 0.06550023, 0.05163103,
       0.06425988, 0.03407733, 0.06851953, 0.06851953, 0.03407733,
       0.06425988, 0.05163103, 0.06550023, 0.04040751, 0.0349968 ,
       0.14060763])

This same circuit can also be simulated without noise.

In [19]:
ideal_sim = QtreeSimulator()
amplitudes = ideal_sim.simulate_batch(circ, num_qubits)
ideal_probs = np.abs(amplitudes)**2
ideal_probs

array([0.26962495, 0.03924902, 0.03924902, 0.02445963, 0.03924902,
       0.02445963, 0.02445963, 0.03924902, 0.03924902, 0.02445963,
       0.02445963, 0.03924902, 0.02445963, 0.03924902, 0.03924902,
       0.26962495])

We can check how well our simulation approximated the exact noisy state by running the same circuit with a density matrix simulator and then calculating the fidelity between the exact and approximate noisy states.

In [26]:
import qiskit.providers.aer.noise as noise
from qiskit.providers.aer import AerSimulator

# Create the same noise model with the same channels and noisy gates in Qiskits framework
depol_chan_qiskit_1Q = noise.depolarizing_error(param = 0.003, num_qubits = 1)
depol_chan_qiskit_2Q = noise.depolarizing_error(param = 0.03, num_qubits = 2)

noise_model_qiskit = noise.NoiseModel()
noise_model_qiskit.add_all_qubit_quantum_error(depol_chan_qiskit_1Q, ['rz', 'rx', 'h'])
noise_model_qiskit.add_all_qubit_quantum_error(depol_chan_qiskit_2Q, ['cx'])

# Take the Qiskit circuit we created before, and add some final touches to it. These do not effect any measurement outcomes 
qiskit_circ = qiskit_comp.circuit

# Qiskit uses little endian notation by default, and Qtensor uses big endian, so we change the Qiskit circuit to match qtensor
qiskit_circ = qiskit_circ.reverse_bits()
qiskit_circ.save_density_matrix()
qiskit_circ.measure_all(add_bits = False)

# Simulate the circuit
backend = AerSimulator(method='density_matrix', noise_model=noise_model_qiskit, fusion_enable=False, fusion_verbose=True)
result = backend.run(qiskit_circ).result()

# The stochastic noisy simulation gives us a probability density vector, so we need to save the diagonal elements of the density matrix for our comparison
qiskit_probs = np.diagonal(np.asarray(result.results[0].data.density_matrix).real)
qiskit_probs

array([0.13403107, 0.05506911, 0.05646327, 0.0514643 , 0.05139734,
       0.0499991 , 0.04779918, 0.05377664, 0.05377664, 0.04779918,
       0.0499991 , 0.05139734, 0.0514643 , 0.05646327, 0.05506911,
       0.13403107])

In [24]:
# The fidelity is defined for probability amplitudes, so we need to take the square root of our probability density vectors
A = np.sqrt(qtensor_probs)
B = np.sqrt(qiskit_probs)
fidelity = np.dot(A, B)**2
fidelity

0.9851105052526412

We can increase our fidelity by increasing the number of circuits in the ensemble.

In [25]:
num_circs = 100
noise_sim.simulate_batch_ensemble(circ, num_circs, num_qubits)
qtensor_probs = noise_sim.normalized_ensemble_probs
A = np.sqrt(qtensor_probs)
B = np.sqrt(qiskit_probs)
fidelity = np.dot(A, B)**2
fidelity

0.9954629065353349